#**CHAPTER 6.REBALANCE COMMITTEE**
---

##REFERENCE

https://chatgpt.com/share/699652ec-6544-8012-818a-c1d3d415c384

##0.CONTEXT

**CHAPTER 6 / NOTEBOOK 6 — Portfolio Rebalance as a Committee, Not a Monologue**

A portfolio rebalance sounds deceptively simple when you say it fast: “take the current weights, apply a view, respect constraints, produce trades.” In practice, it is one of those tasks where the math is clean and the organization is messy. The decision is never only about expected return. It is simultaneously about risk posture, transaction costs, feasibility, governance, and accountability. The real difficulty is not computing a vector of trades. The real difficulty is making a decision that a team can defend, reproduce, and audit after the fact, when the market moves and someone asks, “Why did you do that?”

This notebook’s architecture solves that institutional problem: **how to turn a rebalance into a controlled, parallel, reviewable process** rather than a single “smart” output. The goal is not to maximize cleverness. The goal is to encode a disciplined workflow: multiple perspectives are expressed in parallel, their outputs are aggregated deterministically, disagreements are measured explicitly, and the system terminates with a clear gate: approve, reject, or escalate to human review. In other words, the architecture is not trying to be a genius trader. It is trying to be a **reliable committee secretary** with perfect memory for what happened, who did what, and why the process stopped.

To understand the point, imagine a real investment team on a rebalance day. The portfolio manager has a calendar invite titled “Monthly Rebalance — 45 minutes.” That title is a lie. The meeting is never 45 minutes, because the portfolio is not just a spreadsheet. It is a living set of exposures, client constraints, internal policies, and market frictions. The team walks in with three different anxieties: the risk person worries about concentration and volatility, the execution person worries about turnover and slippage, and the research person worries about missing the signal and drifting from the strategy’s intention. They can all be correct at the same time. The team’s real task is not to “pick the right answer” but to negotiate a decision that is coherent under constraint.

In a human workflow, the committee usually begins with an intake. Someone states the portfolio universe and current weights; someone confirms constraints: maximum position size, turnover limits, and any “do not touch” rules. Someone provides a market snapshot: a rough estimate of which names are volatile, which are expensive to trade, which have positive or negative signal. Then the meeting splits into perspective-specific conversations. The risk person says: “We’re too heavy in high-vol names; reduce tail exposure.” The execution person says: “You can do that, but you’ll pay; avoid high-impact trades; minimize churn.” The research person says: “But the model is flashing; if we don’t rotate, we’re ignoring our own signal.” A human PM then synthesizes this tension into a plan. Sometimes the plan is easy: all three agree. Sometimes it is not: the arguments conflict, the trades are large, and the decision is pushed to a senior committee.

This notebook implements that exact workflow as a state-driven agent system. The architecture is not a chat. It is a directed process with explicit nodes, a typed state, and deterministic routing. The state is the shared “whiteboard” that the whole team writes on: it includes the portfolio, the constraints, the synthetic market snapshot, the committee’s intermediate reports, and the final decision. The system does not rely on “implicit memory” or hidden context. Every decision is produced by functions that update state in traceable increments. If a node fails, it writes an error into state. If a node succeeds, it writes a trace event into state. This is a governance-first approach: state is not merely data; it is the audit record.

The key architectural move in this notebook is **parallel committee deliberation**. Instead of asking one model to produce “the” rebalance, the system runs three LLM tasks in parallel, each constrained to a single mandate. In human terms: we are assigning three analysts to produce three memos at the same time. The risk analyst produces a “risk-first” trade suggestion, the cost analyst produces a “cost-first” suggestion, and the signal analyst produces a “signal-first” suggestion. Each is forced to output strict JSON, not prose. This matters because the committee outputs are not meant to persuade; they are meant to be merged and tested. A persuasive paragraph is not a trade list. A trade list is an object that can be constrained, scaled, and audited.

Parallel work introduces a second institutional reality: concurrency creates collisions. Two people can speak at once; two memos can mention the same trade; two errors can occur in parallel. The architecture handles this with explicit merge channels in the state schema. Certain state fields are designed to accept multiple updates in the same step (for example, committee reports, trace events, and error messages). This is not an implementation detail; it is a modeling choice: it recognizes that a committee is not a single stream of thought. It is a multi-stream process that must be collected and reconciled without losing information.

Once the committee outputs exist, the system transitions from “creative proposal” to “mechanical aggregation.” This is the heart of the control philosophy: **LLM creativity is bounded to the proposal phase; aggregation is deterministic**. The reducer node is the portfolio manager’s calculator. It takes the three trade suggestions and merges them using fixed weights across perspectives. It then enforces constraints. If trades violate maximum weights, it clips. If turnover is too high, it scales trades down to respect the cap. If the trade list is noisy, it prunes tiny changes. This mirrors how real desks operate: the meeting can be open-ended, but the final blotter must obey rules. The reducer makes sure the output is feasible and consistent with declared constraints, without requiring the LLM to “remember” those rules perfectly.

Then comes the gate. Every institutional process needs a stop/go control point, because teams fail in two ways: they either push decisions through without scrutiny, or they talk forever and never act. The gate is designed to avoid both failure modes. It asks: did any node emit errors? If yes, the system stops and requests human review. If no, did the committee disagree strongly? If disagreement is above a threshold, the system can run a bounded rerun for stability (at most a fixed number of rounds). This is a controlled version of what humans do when they sense uncertainty: “Let’s sanity-check. Let’s re-run the analysis. Let’s see if the recommendation is stable.” The key is boundedness. We never allow infinite loops. The system can re-run once for stability, and then it must stop. That creates a discipline: escalation is not a failure; it is an expected outcome when information is insufficient or when the team cannot converge.

This is the narrative the notebook is teaching: **a rebalance decision is a governance problem first and a numerical problem second**. Without explicit controls, an LLM-based workflow degenerates into a single persuasive output that nobody can reproduce. With controls, the system behaves like a small institution: intake, parallel deliberation, deterministic aggregation, explicit stage gate, and artifact export. The artifacts are not decorative. They are the deliverables that make the workflow reviewable: a run manifest (what code and config was used), a graph spec (what topology was executed), and a final state (what happened and what was decided). In a real organization, these are the documents that protect you when something goes wrong. They let you answer questions like: Which model produced the committee suggestions? What constraints were applied? How many rounds were attempted? Why did the system stop? What errors occurred? What trades were proposed?

The notebook is also honest about what it is not doing. It is not executing trades. It is not claiming alpha. It is not optimizing a full risk model. It is teaching the structural pattern: **parallel committee + aggregation**. That pattern is reusable. You can swap the synthetic portfolio for a real one, swap the synthetic snapshot for a real factor/risk feed, and keep the topology. You can add more committee members (compliance, liquidity, tax) without rewriting the whole system. You can harden constraints and expand audit logs. The design scales because the structure is modular: each node has a single job, the state schema is explicit, and the routing rules are deterministic.

If you picture the architecture as a human team, it looks like this. The intake node is the operations associate who assembles the pack. The committee members are three analysts who write focused memos in parallel. The reducer is the portfolio manager’s spreadsheet that turns memos into a feasible plan. The gate is the investment committee chair who decides whether the plan is approved or needs escalation. And the artifact exporter is the risk-control function that files the run: “This is what we did, this is why, and this is the evidence we can show.”

That is the story this notebook tells: not the story of a model “deciding” a rebalance, but the story of a controlled process that turns multiple partial truths into one accountable output. In institutional finance, that difference is everything.


##1.LIBRARIES AND ENVIRONMENT

**CELL 1/10 — Libraries and environment (why we pin and verify)**

This first cell is the “foundation slab” of the notebook. In Colab, you are not starting from a clean machine. You are starting from a crowded kitchen where multiple chefs already left ingredients on the counter. That’s why we do two things that look boring but are absolutely essential for a classroom-grade workflow: **pin versions** and **verify the environment**.

Pinning means we force specific package versions (requests, numpy, pydantic, httpx, langgraph, langchain, etc.) so every student gets the same behavior. In finance, reproducibility is not a “nice to have.” If a rebalance process produces different outputs on different machines, you cannot defend it. A portfolio committee cannot rely on “it worked on my laptop.” This is why we print `CONFIG` and `VERSIONS`. You want to know exactly what model is locked, what thresholds are set, and what libraries were loaded.

The notebook also handles a known Colab problem: preinstalled packages often conflict with your pins. The pip warning you saw earlier is exactly that. The fix is: **install with force-reinstall** and then run `pip check`. If `pip check` complains, you want to see it immediately. Silent dependency conflicts lead to confusing runtime errors later (for example, an API call failing because a library downgraded a dependency behind your back).

The `VERSION_MISMATCH` dictionary is a simple but powerful control. It tells you: “Did the pins actually apply?” If it is empty, your environment matches the intended teaching setup. If it is not empty, you should restart runtime once, because Python can keep old modules in memory even after pip installs new ones. This is not theoretical; it’s the most common source of “why does it still behave like the old version?”

Finally, we set deterministic seeds (`random.seed`, `PYTHONHASHSEED`). Even though we use an LLM later, we still want everything else to be deterministic: synthetic portfolio generation, synthetic features, and aggregation logic. The point is: you can vary the LLM, but you should not vary the plumbing.

So Cell 1 is not just “install stuff.” It is your first governance control: **a reproducible, inspectable environment** that makes every run explainable.


In [1]:
# CELL 1/10 — Install + core imports (Colab hardened + remove langgraph-prebuilt conflicts)

# 1) Remove the conflicting prebuilt package if present (non-fatal if absent)
!pip -q uninstall -y langgraph-prebuilt || true

# 2) Install pinned stack aligned with Colab constraints
!pip -q install --upgrade --force-reinstall \
  "requests==2.32.4" \
  "numpy==2.0.2" \
  "pydantic==2.12.3" \
  "httpx==0.28.1" "httpcore==1.0.5" \
  "langgraph==0.2.39" "langchain==0.3.14" "langchain-core==0.3.40" \
  "anthropic>=0.34.0"

# sanity: dependency integrity (review if it complains)
!pip -q check || true

import os, json, uuid, time, random, hashlib, platform, re
import datetime as _dt
from typing import TypedDict, Literal, Dict, Any, List, Optional, Callable, Tuple
from typing_extensions import Annotated
import operator

import httpx
from google.colab import userdata
from IPython.display import HTML, display
from langgraph.graph import StateGraph, END

random.seed(7)
os.environ["PYTHONHASHSEED"] = "7"

import importlib.metadata as md
def _ver(pkg: str) -> str:
    try:
        return md.version(pkg)
    except Exception:
        return "missing"

CONFIG: Dict[str, Any] = {
    "project": "AA-FIN-LG-2026",
    "notebook": "N6 — Portfolio Rebalance: Parallel Committee + Aggregation",
    "model": "claude-haiku-4-5-20251001",
    "temperature": 0.0,
    "max_tokens": 700,
    "seed": 7,
    "max_rounds": 2,
    "disagreement_threshold": 0.18,
    "committee_perspectives": ["risk", "cost", "signal"],
}

VERSIONS = {
    "python": platform.python_version(),
    "platform": platform.platform(),
    "requests": _ver("requests"),
    "numpy": _ver("numpy"),
    "pydantic": _ver("pydantic"),
    "httpx": _ver("httpx"),
    "httpcore": _ver("httpcore"),
    "langgraph": _ver("langgraph"),
    "langchain": _ver("langchain"),
    "langchain-core": _ver("langchain-core"),
    "anthropic": _ver("anthropic"),
    "langgraph-prebuilt": _ver("langgraph-prebuilt"),  # should become "missing"
}

print("CONFIG:", json.dumps(CONFIG, indent=2))
print("VERSIONS:", json.dumps(VERSIONS, indent=2))
print("UTC_NOW:", _dt.datetime.now(_dt.timezone.utc).isoformat())

EXPECTED = {
    "requests": "2.32.4",
    "numpy": "2.0.2",
    "pydantic": "2.12.3",
    "httpx": "0.28.1",
    "httpcore": "1.0.5",
    "langgraph": "0.2.39",
    "langchain": "0.3.14",
    "langchain-core": "0.3.40",
}
mismatch = {k: {"expected": EXPECTED[k], "got": VERSIONS.get(k)} for k in EXPECTED if VERSIONS.get(k) != EXPECTED[k]}
print("VERSION_MISMATCH:", json.dumps(mismatch, indent=2))


CONFIG: {
  "project": "AA-FIN-LG-2026",
  "notebook": "N6 \u2014 Portfolio Rebalance: Parallel Committee + Aggregation",
  "model": "claude-haiku-4-5-20251001",
  "temperature": 0.0,
  "max_tokens": 700,
  "seed": 7,
  "max_rounds": 2,
  "disagreement_threshold": 0.18,
  "committee_perspectives": [
    "risk",
    "cost",
    "signal"
  ]
}
VERSIONS: {
  "python": "3.12.12",
  "platform": "Linux-6.6.105+-x86_64-with-glibc2.35",
  "requests": "2.32.4",
  "numpy": "2.0.2",
  "pydantic": "2.12.3",
  "httpx": "0.28.1",
  "httpcore": "1.0.5",
  "langgraph": "0.2.39",
  "langchain": "0.3.14",
  "langchain-core": "0.3.40",
  "anthropic": "0.82.0",
  "langgraph-prebuilt": "missing"
}
UTC_NOW: 2026-02-18T23:08:46.405196+00:00
VERSION_MISMATCH: {}


##2.VISUALIZATION STANDARDS

###2.1.OVERVIEW

**CELL 2/10 — Visualization standard (why the graph is mandatory)**

Cell 2 is the “map on the wall.” In agentic systems, especially in finance, you cannot teach or review a workflow if the topology is invisible. People will misunderstand what runs first, what runs in parallel, and where loops occur. That misunderstanding becomes operational risk. So we make visualization a first-class requirement: **every notebook must render the LangGraph topology**.

The key is the hardened Mermaid ESM renderer. We pin Mermaid to a specific version (`10.6.1`) and load it through a module import inside Colab. This avoids external rendering services and avoids version drift. If you rely on some third-party endpoint, you introduce a hidden dependency that can fail during a lecture. If you rely on “whatever Mermaid version Colab happens to have,” you risk diagrams that render differently or break after an update. Pinned versions are governance.

The helper `display_langgraph_mermaid(compiled_graph)` does two jobs: it extracts the mermaid string from the compiled graph and then renders it in the notebook. We also return the mermaid string so we can export it later as part of the audit artifacts. That mermaid is not decorative. It is evidence of the workflow that ran.

This notebook’s topology matters because it introduces the new architectural dimension: **parallel committee deliberation**. The diagram must show: entry into intake, fan-out into multiple committee_member branches, merge back into committee_reduce, then gate, then either loop (bounded) or END. If a student sees a serial chain, they will think it is “one model doing everything.” If they see the map-reduce structure, they understand the institutional metaphor: multiple analysts in parallel, then a portfolio manager aggregating.

The “strict security level” in Mermaid is also not random. It prevents unsafe HTML or scripts from being injected into the visualization. In governed workflows, you treat rendering as an output surface that should be hardened, not as a toy.

So Cell 2 is a teaching control: it forces everyone to see the same structure, and it makes the architecture reviewable at a glance. In finance terms, it is like insisting that every process has an up-to-date process flow diagram attached to the policy.


###2.2.CODE AND IMPLEMENTATION

In [2]:
# CELL 2/10 — Visualization Standard v1: Hardened Mermaid ESM renderer + graph display helper

MERMAID_VERSION = "10.6.1"  # pinned unless discussed

def _escape_html(s: str) -> str:
    return (s.replace("&", "&amp;")
             .replace("<", "&lt;")
             .replace(">", "&gt;"))

def render_mermaid_locally(mermaid_code: str, *, height_px: int = 560) -> None:
    """
    Hardened Mermaid renderer for Colab using Mermaid ESM from a pinned version.
    - Avoids external services
    - Uses module import from unpkg
    - Sanitizes and renders deterministically
    """
    safe = _escape_html(mermaid_code)
    diagram_id = f"mmd-{abs(hash(mermaid_code)) % 10_000_000}"

    html = f"""
<div style="border:1px solid rgba(0,0,0,0.12); border-radius:12px; padding:12px; overflow:auto; height:{height_px}px;">
  <div id="{diagram_id}" class="mermaid">
{safe}
  </div>
</div>

<script type="module">
  import mermaid from "https://unpkg.com/mermaid@{MERMAID_VERSION}/dist/mermaid.esm.min.mjs";
  mermaid.initialize({{
    startOnLoad: false,
    securityLevel: "strict",
    theme: "default",
    flowchart: {{ curve: "linear" }},
    maxTextSize: 200000
  }});
  const el = document.getElementById("{diagram_id}");
  const code = el.textContent;
  const {{ svg }} = await mermaid.render("{diagram_id}-svg", code);
  el.innerHTML = svg;
</script>
"""
    display(HTML(html))

def display_langgraph_mermaid(compiled_graph: Any) -> str:
    """
    Mandatory visualization helper.
    Returns the mermaid string and renders it in-notebook.
    """
    # LangGraph compiled graphs typically expose get_graph().draw_mermaid()
    mmd = compiled_graph.get_graph().draw_mermaid()
    render_mermaid_locally(mmd)
    return mmd

print("Mermaid pinned version:", MERMAID_VERSION)


Mermaid pinned version: 10.6.1


##3.STATE SCHEMA

###3.1.OVERVIEW

**CELL 3/10 — Typed state + merge channels + AgentNode abstraction (the control core)**

Cell 3 is the “constitution” of the whole system. It defines what the workflow is allowed to know and what it is allowed to change. We implement that with an explicit `TypedDict` called `RebalanceState`. This is not just Python typing for aesthetics. It is a governance mechanism: it forces the notebook to treat state as a structured object, not as a bag of hidden variables.

The biggest lesson in this notebook is that parallel workflows require explicit merge logic. In a parallel committee, multiple branches may update the same conceptual field in the same step. For example, three committee members produce three reports at once. If you store those in a normal list field, LangGraph will throw an error because it does not know how to merge concurrent updates. That is why we use `Annotated[List[...], operator.add]` for `committee_reports`, and we do the same for `errors` and `trace`.

This is the “institutional” part: multiple people can file notes simultaneously, and you need a deterministic rule to combine them. Here the rule is simple: **concatenate lists**. That means each node returns only a delta (for example, one report), and LangGraph merges them into the full list.

The `AgentNode` abstraction is required by your project rules and is also the right mental model. Each node is a wrapper around a pure function that maps `state -> patch`. The wrapper adds two controls automatically: (1) a trace event on success, and (2) an error capture on failure. Importantly, the wrapper emits only deltas for mergeable keys. That prevents concurrency collisions and keeps updates auditable.

This design is the opposite of a “chatbot.” A chatbot hides its process inside text. Here the process is explicit: node name, timing, success/failure, and updates are visible in state. When something goes wrong, you don’t guess; you read the error list and the trace log.

Cell 3 therefore establishes the core discipline of the project: **state drives routing, routing drives execution, execution produces artifacts**. Everything else in the notebook is built on this foundation.


###3.2.CODE AND IMPLEMENTATION

In [9]:
# CELL 3/10 — State schema (TypedDict) + audit helpers + AgentNode abstraction (parallel-safe)

import operator
from typing_extensions import Annotated

def utc_now() -> str:
    return _dt.datetime.now(_dt.timezone.utc).isoformat()

def stable_hash(obj: Any) -> str:
    b = json.dumps(obj, sort_keys=True, ensure_ascii=False).encode("utf-8")
    return hashlib.sha256(b).hexdigest()

def write_json(path: str, obj: Any) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2, ensure_ascii=False)

class RebalanceState(TypedDict, total=False):
    # governance + metadata
    run_id: str
    ts_utc: str
    config: Dict[str, Any]
    versions: Dict[str, Any]

    # portfolio + inputs
    universe: List[str]
    holdings: Dict[str, float]          # current weights
    target_policy: Dict[str, Any]       # policy definition
    constraints: Dict[str, Any]         # max weight, turnover cap, etc.
    market_snapshot: Dict[str, Any]     # synthetic risk/cost/signal features

    # committee process
    round: int
    committee_perspectives: List[str]
    committee_expected: int
    committee_reports: Annotated[List[Dict[str, Any]], operator.add]  # MAP merge channel
    committee_scores: Dict[str, Any]

    # outputs
    proposed_trades: Dict[str, float]   # delta weights
    rebalance_decision: Dict[str, Any]  # final decision + rationale
    final_decision: Literal["APPROVE", "HUMAN_REVIEW", "REJECT"]

    # control + audit (parallel-safe merge channels)
    errors: Annotated[List[str], operator.add]
    trace: Annotated[List[Dict[str, Any]], operator.add]

class AgentNode:
    """
    Required abstraction.
    IMPORTANT: For keys that can be updated in parallel steps (errors/trace/committee_reports),
    the node must emit ONLY deltas, letting LangGraph merge them via Annotated operators.
    """
    def __init__(self, name: str, fn: Callable[[RebalanceState], Dict[str, Any]]):
        self.name = name
        self.fn = fn

    def __call__(self, state: RebalanceState) -> Dict[str, Any]:
        t0 = time.time()
        try:
            patch = self.fn(state)
            dt_ms = int((time.time() - t0) * 1000)
            evt = {"ts_utc": utc_now(), "node": self.name, "ms": dt_ms, "ok": True}
            return {**patch, "trace": [evt]}  # delta only (merged)
        except Exception as e:
            dt_ms = int((time.time() - t0) * 1000)
            msg = f"{self.name} error: {type(e).__name__}: {str(e)}"
            evt = {"ts_utc": utc_now(), "node": self.name, "ms": dt_ms, "ok": False, "error": msg}
            return {"errors": [msg], "trace": [evt]}  # deltas only (merged)

print("Cell 3 ready: parallel-safe state + AgentNode.")


Cell 3 ready: parallel-safe state + AgentNode.


##4.SYNTHETIC PORTFOLIO

###4.1.OVERVIEW

**CELL 4/10 — Synthetic portfolio and synthetic snapshot (fast and deterministic)**

Cell 4 creates the “world” the system will operate on. We use synthetic data on purpose. In a teaching setting, external market data introduces noise, API failures, and hidden assumptions. Synthetic data lets us control the environment and focus on architecture. The objective is not realism in prices; the objective is realism in workflow structure.

We generate a fixed universe of tickers and random starting weights. Because we set a seed, every run produces the same starting portfolio. That is important because it makes comparisons meaningful. If the portfolio changes randomly between runs, students can’t tell whether differences come from the architecture or the inputs.

Then we create a synthetic market snapshot with three features per asset: **risk**, **cost**, and **signal**. These map directly to the committee perspectives in this notebook. Risk is treated as “higher is worse” (think volatility proxy). Cost is treated as “higher is worse” (think impact or slippage proxy). Signal is treated as “higher is better” (think alpha or score proxy). These are not full finance models, but they are enough to create a real tension: the asset with the best signal might also be costly to trade, and the asset with the lowest risk might have a weak signal.

We also define constraints and a simple policy stub. Constraints include max weight, min weight, a turnover cap, and a minimum trade size. These constraints are what turns “a suggestion” into “an implementable plan.” In real portfolio work, constraints are not optional. Without them, you are writing fiction.

The design choice here is very intentional: the LLM will propose trades, but the deterministic reducer will enforce constraints. That division of labor is only meaningful if constraints exist and are explicit. Cell 4 is therefore the first half of the “control story”: it creates the portfolio and the rules of the game.

This cell is also optimized for speed. Everything runs instantly. That matters because the architecture depends on quick iteration. Students should be able to rerun the process, adjust parameters like turnover cap, and observe how the workflow responds, without waiting for heavy computations.


###4.2.CODE AND IMPLEMENTATION

In [4]:
# CELL 4/10 — Synthetic portfolio + synthetic market snapshot (fast, classroom-safe)

def make_synthetic_portfolio(seed: int = 7) -> Tuple[List[str], Dict[str, float]]:
    rnd = random.Random(seed)
    universe = ["AAPL", "MSFT", "NVDA", "AMZN", "GOOGL", "JPM", "XOM", "UNH"]
    raw = [rnd.random() for _ in universe]
    s = sum(raw)
    w = {a: v / s for a, v in zip(universe, raw)}
    return universe, w

def make_synthetic_snapshot(universe: List[str], seed: int = 7) -> Dict[str, Any]:
    rnd = random.Random(seed + 101)
    snap: Dict[str, Any] = {"features": {}}
    for a in universe:
        # risk: higher is worse; cost: higher is worse; signal: higher is better
        risk = round(0.10 + 0.35 * rnd.random(), 4)     # proxy: vol
        cost = round(0.0005 + 0.004 * rnd.random(), 5)  # proxy: impact per 1.0 turnover
        signal = round(-1.0 + 2.0 * rnd.random(), 4)    # proxy: z-score alpha
        snap["features"][a] = {"risk": risk, "cost": cost, "signal": signal}
    return snap

def default_constraints() -> Dict[str, Any]:
    return {
        "max_weight": 0.22,
        "min_weight": 0.00,
        "turnover_cap": 0.35,         # L1 turnover cap on sum(|dw|)
        "min_trade_abs": 0.0025,      # prune tiny trades
    }

def default_policy() -> Dict[str, Any]:
    return {"type": "tilt_equal_weight", "tilt_strength": 0.22}

u, h = make_synthetic_portfolio(CONFIG["seed"])
snap = make_synthetic_snapshot(u, CONFIG["seed"])
print("Universe:", u)
print("Holdings sum:", round(sum(h.values()), 8))
print("Sample features:", {k: snap["features"][k] for k in u[:3]})


Universe: ['AAPL', 'MSFT', 'NVDA', 'AMZN', 'GOOGL', 'JPM', 'XOM', 'UNH']
Holdings sum: 1.0
Sample features: {'AAPL': {'risk': 0.1457, 'cost': 0.00376, 'signal': 0.7609}, 'MSFT': {'risk': 0.2348, 'cost': 0.00161, 'signal': -0.6082}, 'NVDA': {'risk': 0.1283, 'cost': 0.00136, 'signal': -0.596}}


##5.LLM WRAPPER

###5.1.OVERVIEW

**CELL 5/10 — LLM wrapper (minimal, correct API shape, governance-friendly)**

Cell 5 is where the notebook becomes genuinely agentic: it can call the LLM. The most important thing to understand is that we are not using a large framework with hidden memory. We are using a minimal HTTP client with explicit inputs. That is deliberate. In finance, you want to reduce hidden behavior. “Magic” clients that silently store context or adjust prompts are convenient, but they make audits hard.

This wrapper enforces three critical controls. First, it retrieves the API key using `userdata.get("ANTHROPIC_API_KEY")` in all caps. That’s the Colab secret mechanism. The notebook refuses to run if the key is missing. Silent fallbacks are dangerous because they can lead to confusing “why did it fail?” moments.

Second, the wrapper uses the correct Messages API shape: **system is a top-level field**, and `messages` includes only user/assistant roles. This matters because incorrect request shape is a common source of 400 errors. This is not “complexity for complexity’s sake.” It is a correctness fix.

Third, we keep the wrapper small and testable. It takes `system`, `user_content`, model name, temperature, and max tokens. It returns plain text. That’s it. No tool calls, no streaming, no retries. This is a classroom notebook; we prefer clarity over feature volume.

Finally, we include a strict JSON extraction helper. The committee nodes are instructed to output JSON only, but models sometimes add extra text. The parser searches for a `{...}` block and parses it. If no JSON is found, it raises an error, which will be caught by `AgentNode` and written into state. That is governance: failures become visible state, not hidden exceptions.

So Cell 5 defines the “LLM as a controlled tool.” It is not a conversational partner. It is a service that returns a structured proposal, and every call is explicit, parameterized, and auditable.


###5.2.CODE AND IMPLEMENTATION

In [23]:
# CELL 5/10 — Minimal Anthropic Messages API wrapper (correct request shape) + safe JSON extraction

def get_api_key() -> str:
    key = userdata.get("ANTHROPIC_API_KEY")  # ALL CAPS, required
    if not key or not isinstance(key, str):
        raise RuntimeError('Missing Colab secret: userdata.get("ANTHROPIC_API_KEY")')
    return key.strip()

def anthropic_text(*,
                   system: str,
                   user_content: str,
                   model: str,
                   temperature: float,
                   max_tokens: int) -> str:
    """
    Minimal HTTP call to Anthropic /v1/messages.

    IMPORTANT:
    - 'system' is a TOP-LEVEL field in the payload (not a message role).
    - messages contain only user/assistant roles.
    """
    headers = {
        "x-api-key": get_api_key(),
        "anthropic-version": "2023-06-01",
        "content-type": "application/json",
    }

    payload = {
        "model": model,
        "max_tokens": int(max_tokens),
        "temperature": float(temperature),
        "system": system,
        "messages": [{"role": "user", "content": user_content}],
    }

    with httpx.Client(timeout=45.0) as client:
        r = client.post("https://api.anthropic.com/v1/messages", headers=headers, json=payload)
        r.raise_for_status()
        data = r.json()

    parts = data.get("content", [])
    text = "".join([p.get("text", "") for p in parts if p.get("type") == "text"])
    return text

_JSON_BLOCK_RE = re.compile(r"\{.*\}", re.DOTALL)

def parse_json_from_text(text: str) -> Dict[str, Any]:
    """
    Strict-ish JSON extraction: find the first {...} block and parse.
    """
    m = _JSON_BLOCK_RE.search(text)
    if not m:
        raise ValueError("No JSON object found in model output.")
    return json.loads(m.group(0))

print("Cell 5 ready: Anthropic wrapper uses top-level system field.")


Cell 5 ready: Anthropic wrapper uses top-level system field.


##6.AGENT NODES

###6.1.OVERVIEW

**CELL 6/10 — Agent nodes (human team roles encoded as functions)**

Cell 6 is where the narrative becomes concrete: we define the nodes that behave like members of a rebalance team. The intake node is like the operations associate assembling the pack: it ensures the state contains the portfolio universe, current holdings, constraints, policy, and market snapshot. It also initializes the mergeable lists (`committee_reports`, `errors`, `trace`) to empty lists at the start of a run or round. This matters because mergeable channels should start clean; otherwise you mix reports from different rounds.

The committee_member node is the key “LLM usage” node. It represents one human analyst with a single mandate. We do not ask the model to do everything. We ask it to propose trades under one perspective: risk, cost, or signal. The prompt is structured, includes the holdings, the features, the constraints, and an output schema. The model must return strict JSON. We also print `[LLM CALL]` and a short preview of raw output so students can see the call happened.

The reducer node is the portfolio manager’s discipline. It does not call the LLM. It takes the committee reports and merges them deterministically using fixed weights across perspectives. Then it enforces constraints: clipping to max/min weights, normalizing, pruning tiny trades, and scaling trades if turnover exceeds the cap. This is the governance principle: proposals may be “creative,” but the final plan must be mechanically feasible.

The gate node is the committee chair. It checks for errors first. If errors exist, we escalate to HUMAN_REVIEW. If no errors, it checks disagreement. If disagreement is high and rounds remain, it triggers a bounded rerun. Otherwise it approves. This creates a controlled stopping rule: no infinite debates, no infinite loops.

So Cell 6 encodes a real team: intake, three analysts, a portfolio manager aggregator, and a chair who decides when to stop. It is not “prompt engineering.” It is **organizational design expressed as a state machine**.


###6.2.CODE AND IMPLEMENTATION

In [25]:
# CELL 6/10 — Agent nodes: intake -> MAP (parallel committee) -> REDUCE (aggregation) -> gate -> END
# (Parallel-safe merge channels + explicit LLM visibility in committee_member; uses corrected Cell 5 wrapper)

def intake_fn(state: RebalanceState) -> Dict[str, Any]:
    """
    Intake initializes deterministic run metadata and synthetic inputs (if absent).
    NOTE: We initialize merge-channel lists explicitly as empty lists.
    """
    run_id = state.get("run_id") or str(uuid.uuid4())

    base: RebalanceState = {
        "run_id": run_id,
        "ts_utc": utc_now(),
        "config": CONFIG,
        "versions": VERSIONS,
        "round": int(state.get("round", 1)),
        "committee_perspectives": list(CONFIG["committee_perspectives"]),
        "committee_expected": len(CONFIG["committee_perspectives"]),
        "committee_reports": [],  # merged list, start empty each round
        "errors": [],             # merged list, start empty each round
        "trace": [],              # merged list, start empty each round
    }

    # attach synthetic inputs if not present
    if "universe" not in state:
        u, h = make_synthetic_portfolio(CONFIG["seed"])
        base["universe"] = u
        base["holdings"] = h
        base["constraints"] = default_constraints()
        base["target_policy"] = default_policy()
        base["market_snapshot"] = make_synthetic_snapshot(u, CONFIG["seed"])
    else:
        for k in ["universe", "holdings", "constraints", "target_policy", "market_snapshot"]:
            if k in state:
                base[k] = state[k]

    return base

intake = AgentNode("intake", intake_fn)

def committee_member_fn(state: RebalanceState) -> Dict[str, Any]:
    """
    One committee member (perspective) proposes delta-weights under constraints.
    Returns a ONE-ELEMENT list for committee_reports so parallel branches can merge via operator.add.
    """
    perspective = state.get("committee_current_perspective")
    if perspective not in CONFIG["committee_perspectives"]:
        raise ValueError("Invalid or missing committee_current_perspective")

    holdings = state["holdings"]
    feats = state["market_snapshot"]["features"]
    policy = state["target_policy"]
    cons = state["constraints"]

    sys = "You are a portfolio rebalance committee member. Output STRICT JSON only. No prose outside JSON."
    user_obj = {
        "task": "Propose delta-weights (trades) under your assigned perspective, consistent with constraints.",
        "perspective": perspective,
        "holdings": holdings,
        "features": feats,
        "policy": policy,
        "constraints": cons,
        "output_schema": {
            "perspective": "risk|cost|signal",
            "proposed_trades": {"TICKER": "delta_weight_float"},
            "notes": "short string",
            "metrics": {"turnover": "float", "top_changes": ["TICKER", "..."]},
        },
        "rules": [
            "Return STRICT JSON only.",
            "Sum of trades should be ~0 (self-financing).",
            "Respect max_weight/min_weight after applying trades.",
            "Keep turnover reasonable; aggregator will enforce hard cap.",
            "If unsure, return smaller trades.",
        ],
    }

    # --- visibility: prove the LLM is called per parallel branch ---
    print(f"[LLM CALL] committee_member perspective={perspective} model={CONFIG['model']}")

    txt = anthropic_text(
        system=sys,
        user_content=json.dumps(user_obj, ensure_ascii=False),
        model=CONFIG["model"],
        temperature=CONFIG["temperature"],
        max_tokens=CONFIG["max_tokens"],
    )

    print(f"[LLM RAW PREVIEW] {perspective}:", txt[:220].replace("\n", " "), "...")

    report = parse_json_from_text(txt)
    report["perspective"] = perspective

    if "proposed_trades" not in report or not isinstance(report["proposed_trades"], dict):
        raise ValueError("committee report missing proposed_trades dict")

    return {"committee_reports": [report]}  # delta list (merged)

committee_member = AgentNode("committee_member", committee_member_fn)

def reduce_fn(state: RebalanceState) -> Dict[str, Any]:
    """
    Aggregation:
      - Combine committee proposals using deterministic weights
      - Enforce constraints (max/min weights, turnover cap)
      - Score disagreement for a bounded stability rerun
    """
    reports = state.get("committee_reports", [])
    expected = int(state["committee_expected"])

    if len(reports) < expected:
        return {}  # not ready yet

    universe = state["universe"]
    holdings = state["holdings"]
    cons = state["constraints"]

    w = {"risk": 0.40, "cost": 0.25, "signal": 0.35}

    per: Dict[str, Dict[str, float]] = {}
    for r in reports:
        p = str(r.get("perspective", ""))
        vec = {k: float(v) for k, v in r.get("proposed_trades", {}).items()}
        per[p] = vec
    for p in CONFIG["committee_perspectives"]:
        per.setdefault(p, {})

    agg: Dict[str, float] = {a: 0.0 for a in universe}
    for p in CONFIG["committee_perspectives"]:
        wp = float(w.get(p, 0.0))
        for a in universe:
            agg[a] += wp * float(per[p].get(a, 0.0))

    s = sum(agg.values())
    if abs(s) > 1e-12:
        adj = s / len(universe)
        for a in universe:
            agg[a] -= adj

    min_abs = float(cons["min_trade_abs"])
    for a in universe:
        if abs(agg[a]) < min_abs:
            agg[a] = 0.0

    max_w = float(cons["max_weight"])
    min_w = float(cons["min_weight"])

    def clip_and_norm(weights: Dict[str, float]) -> Dict[str, float]:
        clipped = {a: min(max(weights[a], min_w), max_w) for a in universe}
        tot = sum(clipped.values())
        if tot <= 0:
            ew = 1.0 / len(universe)
            return {a: ew for a in universe}
        return {a: clipped[a] / tot for a in universe}

    post = {a: holdings[a] + agg[a] for a in universe}
    post = clip_and_norm(post)
    trades = {a: post[a] - holdings[a] for a in universe}

    turnover = sum(abs(trades[a]) for a in universe)
    cap = float(cons["turnover_cap"])
    scale = 1.0
    if turnover > cap and turnover > 1e-12:
        scale = cap / turnover
        trades = {a: trades[a] * scale for a in universe}
        post2 = {a: holdings[a] + trades[a] for a in universe}
        post2 = clip_and_norm(post2)
        trades = {a: post2[a] - holdings[a] for a in universe}
        turnover = sum(abs(trades[a]) for a in universe)

    aligned = []
    for p in CONFIG["committee_perspectives"]:
        aligned.append({a: float(per[p].get(a, 0.0)) for a in universe})

    d = []
    for i in range(len(aligned)):
        for j in range(i + 1, len(aligned)):
            d.append(sum(abs(aligned[i][a] - aligned[j][a]) for a in universe))
    disagreement = float(sum(d) / max(1, len(d)))

    scores = {
        "turnover": float(turnover),
        "turnover_cap": cap,
        "turnover_scale": float(scale),
        "disagreement_l1_avg": disagreement,
        "expected_reports": expected,
        "received_reports": len(reports),
    }

    return {"proposed_trades": trades, "committee_scores": scores}

reduce_node = AgentNode("committee_reduce", reduce_fn)

def gate_fn(state: RebalanceState) -> Dict[str, Any]:
    """
    Decision gate:
      - if any errors -> HUMAN_REVIEW
      - if high disagreement and rounds remaining -> rerun once (bounded)
      - else APPROVE
    """
    errs = state.get("errors", [])
    scores = state.get("committee_scores", {})
    disagreement = float(scores.get("disagreement_l1_avg", 0.0))
    r = int(state.get("round", 1))

    if errs:
        decision = {
            "status": "HUMAN_REVIEW",
            "reason": "Errors encountered in committee workflow.",
            "errors": errs[:10],
            "scores": scores,
        }
        return {"rebalance_decision": decision, "final_decision": "HUMAN_REVIEW"}

    if disagreement >= float(CONFIG["disagreement_threshold"]) and r < int(CONFIG["max_rounds"]):
        decision = {
            "status": "HUMAN_REVIEW",
            "reason": "High committee disagreement; rerun once for stability (bounded).",
            "scores": scores,
            "next_round": r + 1,
        }
        return {"rebalance_decision": decision, "final_decision": "HUMAN_REVIEW", "round": r + 1}

    decision = {
        "status": "APPROVE",
        "reason": "Committee aggregated proposal meets constraints; disagreement within threshold.",
        "scores": scores,
        "proposed_trades": state.get("proposed_trades", {}),
        "notes": "Trades are delta-weights; execution planning is out of scope for N6.",
    }
    return {"rebalance_decision": decision, "final_decision": "APPROVE"}

gate = AgentNode("gate", gate_fn)

print("Cell 6 ready: intake + committee_member(LLM) + reduce + gate.")


Cell 6 ready: intake + committee_member(LLM) + reduce + gate.


##7.BUILDING THE GRAPH

###7.1.OVERVIEW

**CELL 7/10 — Building the graph (the topology is the lesson)**

Cell 7 turns the nodes into a real workflow using LangGraph. This is where the notebook earns its title: “parallel committee + aggregation.” The topology is not incidental. It is the teaching artifact.

We start with a `fanout_router` that returns a list of `Send` objects. Each Send targets the committee_member node and carries a payload. The important lesson from your debugging is that parallel branches must receive the full state snapshot. If you send only the “perspective” field, the committee_member node will not have holdings, constraints, or anything else. So the router creates a payload from the current state and adds `committee_current_perspective`. That is how we simulate “same pack, different analyst.”

Then we connect the flow: intake -> fanout -> committee_member -> committee_reduce -> gate -> END. The merge behavior happens automatically because we declared merge channels in the state schema. When three committee_member branches finish, their updates to `committee_reports` are concatenated into one list in the shared state. That is the map-reduce pattern in action.

The gate has conditional routing. If it decides to rerun (based on disagreement and round), it loops back to intake. The loop is bounded by `max_rounds`. That boundedness is not optional; it is a hard control.

Finally, we compile the graph and render it with Mermaid. The diagram must match the topology exactly. This is a governance requirement: the executed process must be visible. In regulated environments, it’s the difference between “we think the process works” and “here is the exact workflow that ran.”

Cell 7 therefore is the bridge from functions to system. It’s where the architecture becomes a topology you can inspect, teach, and audit.


###7.2.CODE AND IMPLEMENTATION

In [26]:
# CELL 7/10 — Build LangGraph topology (parallel MAP via Send) + compile + visualize
# (FIXED: Send payload includes the FULL current state so branches never miss keys)

from langgraph.types import Send  # fan-out primitive

def fanout_router(state: RebalanceState) -> List[Send]:
    sends: List[Send] = []
    for p in CONFIG["committee_perspectives"]:
        payload = dict(state)  # IMPORTANT: branch receives full state snapshot
        payload["committee_current_perspective"] = p
        sends.append(Send("committee_member", payload))
    return sends

def gate_router(state: RebalanceState) -> str:
    dec = state.get("rebalance_decision", {}).get("status")
    if dec == "HUMAN_REVIEW" and not state.get("errors"):
        if int(state.get("round", 1)) > 1 and int(state.get("round", 1)) <= int(CONFIG["max_rounds"]):
            return "fanout"
    return "end"

graph = StateGraph(RebalanceState)

graph.add_node("intake", intake)
graph.add_node("committee_member", committee_member)
graph.add_node("committee_reduce", reduce_node)
graph.add_node("gate", gate)

graph.set_entry_point("intake")

graph.add_conditional_edges("intake", fanout_router, ["committee_member"])
graph.add_edge("committee_member", "committee_reduce")
graph.add_edge("committee_reduce", "gate")
graph.add_conditional_edges("gate", gate_router, {"fanout": "intake", "end": END})

compiled = graph.compile()

mmd = display_langgraph_mermaid(compiled)
print("Mermaid chars:", len(mmd))


Mermaid chars: 536


##8.EXECUTION

###8.1.OVERVIEW

**CELL 8/10 — Execution (what happens when the system runs)**

Cell 8 is where the system becomes “alive.” We create an initial state and invoke the compiled graph. The key concept is that the state is not just inputs; it is the evolving record of the process. We initialize the governance fields: run_id, timestamps, config, versions, round, and the mergeable lists. We also include committee metadata (perspectives, expected report count, empty report list). This makes the run self-contained and explicit.

When execution starts, intake ensures the portfolio and snapshot exist. Then the graph fans out into three parallel committee_member calls. In the notebook output, you see three `[LLM CALL]` lines. That is the “human team” metaphor made visible: three analysts working at the same time.

Each committee_member returns a report. Those reports merge into `committee_reports`. Then committee_reduce runs once with the merged list. It produces deterministic `proposed_trades` and `committee_scores`. Those scores include turnover and disagreement. Then gate uses those scores to decide: approve, reject, or human review. If the system is stable and no errors occur, approval is typical. If the LLM fails, parsing fails, or disagreement is high, the gate escalates. That escalation is not a bug. It is a governance behavior: the system is allowed to stop safely.

Cell 8 also prints the outputs in a way that is easy to inspect: final decision, round, scores, decision object, and top trades. This supports the pedagogical goal: students see both the process and the result. They do not have to dig through hidden logs.

Execution here is not “trading.” It is a controlled simulation of the rebalance decision workflow. The deliverable is a proposed trade vector and a decision status, not an execution report. That boundary is intentional: execution belongs in later notebooks and different architectures.

So Cell 8 teaches the main operational lesson: agentic workflows are not text outputs. They are **state transitions through a graph**, with explicit stopping conditions and visible intermediate artifacts.


###8.2.CODE AND IMPLEMENTATION

In [27]:
# CELL 8/10 — Execute graph (single run) + inspect outputs
# (Now works because committee branches receive full state via Send payload)

initial_state: RebalanceState = {
    "run_id": str(uuid.uuid4()),
    "ts_utc": utc_now(),
    "config": CONFIG,
    "versions": VERSIONS,
    "round": 1,

    # include committee metadata explicitly
    "committee_perspectives": list(CONFIG["committee_perspectives"]),
    "committee_expected": len(CONFIG["committee_perspectives"]),
    "committee_reports": [],

    # merge-channel lists start empty
    "errors": [],
    "trace": [],
}

final_state: RebalanceState = compiled.invoke(initial_state)

print("FINAL_DECISION:", final_state.get("final_decision"))
print("ROUND:", final_state.get("round"))
print("SCORES:", json.dumps(final_state.get("committee_scores", {}), indent=2))
print("DECISION:", json.dumps(final_state.get("rebalance_decision", {}), indent=2))

tr = final_state.get("proposed_trades", {})
if tr:
    top = sorted(tr.items(), key=lambda kv: abs(float(kv[1])), reverse=True)[:8]
    print("TOP TRADES (abs):", top)

print("REPORTS_RECEIVED:", len(final_state.get("committee_reports", [])))
print("TRACE_LEN:", len(final_state.get("trace", [])))
print("ERRORS:", final_state.get("errors", []))


[LLM CALL] committee_member perspective=risk model=claude-haiku-4-5-20251001
[LLM CALL] committee_member perspective=cost model=claude-haiku-4-5-20251001
[LLM CALL] committee_member perspective=signal model=claude-haiku-4-5-20251001
[LLM RAW PREVIEW] risk: ```json {   "perspective": "risk",   "proposed_trades": {     "AAPL": 0.0,     "MSFT": -0.0348,     "NVDA": 0.0,     "AMZN": -0.0272,     "GOOGL": -0.0425,     "JPM": -0.0137,     "XOM": 0.0282,     "UNH": 0.09   },   "n ...
[LLM RAW PREVIEW] cost: ```json {   "perspective": "cost",   "proposed_trades": {     "AAPL": -0.01849,     "MSFT": 0.00839,     "NVDA": -0.02424,     "AMZN": 0.02282,     "GOOGL": -0.03607,     "JPM": -0.01221,     "XOM": 0.03327,     "UNH": 0 ...
[LLM RAW PREVIEW] signal: ```json {   "perspective": "signal",   "proposed_trades": {     "AAPL": 0.0275,     "MSFT": -0.0566,     "NVDA": -0.0850,     "AMZN": -0.0272,     "GOOGL": -0.0850,     "JPM": -0.0137,     "XOM": 0.0275,     "UNH": 0.052 ...
[LLM CALL] committ

##9.AUDIT ARTIFACTS

###9.1.0VERVIEW

**CELL 9/10 — Audit artifacts (what makes this institutional, not a demo)**

Cell 9 exports the required artifacts: `run_manifest.json`, `graph_spec.json`, and `final_state.json`. This is where the notebook becomes “audit-ready.” Without these files, you can show a cool demo, but you cannot support professional review. With these files, you can answer the questions that matter in finance: What ran? With what configuration? On what environment? What did it decide? Why did it stop?

The run manifest captures run_id, timestamp, model lock, configuration, a hash of configuration (so you can detect changes), and an environment fingerprint (Python version, platform, library versions). This is similar to what a real desk might keep as a run record: it tells you the precise setup used to generate the recommendation. If someone changes a threshold later, the config hash changes. That’s governance by design.

The graph spec captures the topology: nodes, edges, merge channels, bounded loop conditions, and the Mermaid diagram string. This is important because the topology is part of the “policy.” If a future version changes routing (for example, adds more committee members), the graph spec changes. You can then compare versions cleanly.

The final state captures everything that happened: inputs, reports, trades, decision, errors, and trace events. The trace is especially important. It turns execution into an audit log: which nodes ran, how long they took, and whether they succeeded or failed. In regulated or institutional contexts, trace logs are the difference between “trust me” and “here is the record.”

Cell 9 therefore is not an add-on. It is a core deliverable. It aligns directly with the project’s governance-first principle: every run produces inspectable evidence. The system is judged not only by output quality, but by **process quality**: reproducibility, reviewability, and accountability.


###9.2.CODE AND IMPLEMENTATION

In [28]:
# CELL 9/10 — Audit artifacts (required): run_manifest.json + graph_spec.json + final_state.json

def env_fingerprint() -> Dict[str, Any]:
    return {
        "python": platform.python_version(),
        "platform": platform.platform(),
        "versions": VERSIONS,
        "cwd": os.getcwd(),
    }

def build_run_manifest(state: RebalanceState) -> Dict[str, Any]:
    cfg = dict(CONFIG)
    return {
        "project": cfg.get("project"),
        "notebook": cfg.get("notebook"),
        "run_id": state.get("run_id"),
        "ts_utc": utc_now(),
        "model_lock": cfg.get("model"),
        "config": cfg,
        "config_hash_sha256": stable_hash(cfg),
        "env": env_fingerprint(),
        "notes": [
            "Synthetic-only inputs.",
            "Parallel committee MAP merges via Annotated[List, operator.add].",
            "All loops bounded by CONFIG['max_rounds'].",
        ],
    }

def build_graph_spec(mermaid: str) -> Dict[str, Any]:
    # We keep this spec explicit and topology-aligned with the code (auditable + stable).
    nodes = [
        {"id": "intake", "type": "AgentNode", "role": "Initialize run + inputs"},
        {"id": "committee_member", "type": "AgentNode", "role": "LLM committee proposal (parallel MAP)"},
        {"id": "committee_reduce", "type": "AgentNode", "role": "Deterministic aggregation (REDUCE)"},
        {"id": "gate", "type": "AgentNode", "role": "Decision gate + bounded rerun or END"},
        {"id": "END", "type": "END", "role": "Explicit termination"},
    ]
    edges = [
        {"from": "intake", "to": "committee_member", "kind": "conditional_send_list", "fn": "fanout_router"},
        {"from": "committee_member", "to": "committee_reduce", "kind": "edge"},
        {"from": "committee_reduce", "to": "gate", "kind": "edge"},
        {"from": "gate", "to": "intake", "kind": "conditional", "label": "fanout", "fn": "gate_router"},
        {"from": "gate", "to": "END", "kind": "conditional", "label": "end", "fn": "gate_router"},
    ]
    return {
        "project": CONFIG["project"],
        "notebook": CONFIG["notebook"],
        "ts_utc": utc_now(),
        "mermaid": mermaid,
        "nodes": nodes,
        "edges": edges,
        "merge_channels": {
            "committee_reports": "Annotated[List[Dict], operator.add]",
            "errors": "Annotated[List[str], operator.add]",
            "trace": "Annotated[List[Dict], operator.add]",
        },
        "bounded_loop": {
            "max_rounds": int(CONFIG["max_rounds"]),
            "loop_condition": "gate_router returns 'fanout' only when round>1 and round<=max_rounds and errors==[]",
        },
    }

# --- write required artifacts ---
if "final_state" not in globals() or not isinstance(final_state, dict):
    raise RuntimeError("final_state not found. Run CELL 8/10 successfully before exporting artifacts.")

if "mmd" not in globals() or not isinstance(mmd, str):
    # fallback: re-derive mermaid (still deterministic)
    mmd = compiled.get_graph().draw_mermaid()

run_manifest = build_run_manifest(final_state)
graph_spec = build_graph_spec(mmd)

write_json("run_manifest.json", run_manifest)
write_json("graph_spec.json", graph_spec)
write_json("final_state.json", final_state)

print("WROTE:", ["run_manifest.json", "graph_spec.json", "final_state.json"])
print("run_id:", final_state.get("run_id"))
print("final_decision:", final_state.get("final_decision"))
print("reports_received:", len(final_state.get("committee_reports", [])))
print("errors_count:", len(final_state.get("errors", [])))


WROTE: ['run_manifest.json', 'graph_spec.json', 'final_state.json']
run_id: af64cd1d-3bed-42cb-8905-ec61b74941ce
final_decision: APPROVE
reports_received: 6
errors_count: 0


##10.AUDIT BUNDLE

###10.1.OVERVIEW

**CELL 10/10 — Playground (bounded experimentation without changing topology)**

Cell 10 is the controlled sandbox. The goal is to let students explore “what if” scenarios without changing the architecture. This is important: in professional practice, you often want to stress a process under different inputs before you rewrite it. You want to test robustness without introducing structural drift.

The playground runs a few scenarios by changing synthetic inputs (seed) and constraints (turnover cap). The topology remains identical. That means any differences in outputs are caused by input conditions, not by code changes. This is a scientific attitude: keep structure fixed while varying conditions.

The helper `summarize_state` prints a compact view: final decision, round, report count, error count, and key metrics like turnover and disagreement. This is how you teach interpretation. Students learn to read the outputs as a controlled record: was the system stable, did the committee disagree, did turnover binding occur?

The helper `run_with_inputs` builds an initial state with explicit portfolio, snapshot, constraints, and governance fields. It then invokes the same compiled graph. This is also a good pattern for future notebooks: you can wrap the compiled graph behind a simple function that takes inputs and returns final state.

The “bounded” principle still applies. We do not allow open-ended parameter sweeps that take minutes in class. We run a small number of scenarios quickly. That keeps the notebook usable under classroom time constraints and reinforces the discipline that experiments should be purposeful.

Pedagogically, the playground is where students internalize the architecture: they see that the committee system is not brittle, that constraints shape outcomes, and that disagreements can trigger review. They also learn the deeper message: the value of the system is not just the trade vector. The value is a stable process that explains itself and produces artifacts.

So Cell 10 closes the notebook in the right way: not with more features, but with a controlled way to learn by experimentation.


###10.2.CODE AND IMPLEMENTATION

In [29]:
# CELL 10/10 — Playground (bounded): run a few what-if scenarios without changing topology

def summarize_state(tag: str, st: RebalanceState) -> None:
    scores = st.get("committee_scores", {})
    print("\n===", tag, "===")
    print("final_decision:", st.get("final_decision"))
    print("round:", st.get("round"))
    print("reports_received:", len(st.get("committee_reports", [])))
    print("errors_count:", len(st.get("errors", [])))
    if scores:
        print("disagreement_l1_avg:", scores.get("disagreement_l1_avg"))
        print("turnover:", scores.get("turnover"), "/", scores.get("turnover_cap"))
    tr = st.get("proposed_trades", {})
    if tr:
        top = sorted(tr.items(), key=lambda kv: abs(float(kv[1])), reverse=True)[:5]
        print("top_trades_abs:", top)

def run_with_inputs(seed: int, *, turnover_cap: Optional[float] = None) -> RebalanceState:
    u, h = make_synthetic_portfolio(seed)
    snap = make_synthetic_snapshot(u, seed)
    cons = default_constraints()
    if turnover_cap is not None:
        cons = dict(cons)
        cons["turnover_cap"] = float(turnover_cap)

    init: RebalanceState = {
        "run_id": str(uuid.uuid4()),
        "ts_utc": utc_now(),
        "config": CONFIG,
        "versions": VERSIONS,
        "round": 1,
        "committee_perspectives": list(CONFIG["committee_perspectives"]),
        "committee_expected": len(CONFIG["committee_perspectives"]),
        "committee_reports": [],
        "errors": [],
        "trace": [],
        # provide full inputs so results vary by scenario deterministically
        "universe": u,
        "holdings": h,
        "constraints": cons,
        "target_policy": default_policy(),
        "market_snapshot": snap,
    }
    return compiled.invoke(init)

# Scenario A: baseline synthetic inputs (seed=7)
st_a = run_with_inputs(7)
summarize_state("Scenario A (seed=7, default constraints)", st_a)

# Scenario B: different synthetic market (seed=21)
st_b = run_with_inputs(21)
summarize_state("Scenario B (seed=21, default constraints)", st_b)

# Scenario C: tighter turnover cap (seed=7, turnover_cap=0.12)
st_c = run_with_inputs(7, turnover_cap=0.12)
summarize_state("Scenario C (seed=7, tighter turnover_cap=0.12)", st_c)

print("\nPlayground complete. Topology unchanged; only inputs/constraints varied.")


[LLM CALL] committee_member perspective=risk model=claude-haiku-4-5-20251001
[LLM CALL] committee_member perspective=cost model=claude-haiku-4-5-20251001
[LLM CALL] committee_member perspective=signal model=claude-haiku-4-5-20251001
[LLM RAW PREVIEW] risk: ```json {   "perspective": "risk",   "proposed_trades": {     "AAPL": 0.0,     "MSFT": -0.0348,     "NVDA": 0.0,     "AMZN": -0.0272,     "GOOGL": -0.0425,     "JPM": 0.0,     "XOM": 0.0,     "UNH": 0.1045   },   "notes" ...
[LLM RAW PREVIEW] cost: ```json {   "perspective": "cost",   "proposed_trades": {     "AAPL": -0.01849,     "MSFT": 0.00839,     "NVDA": -0.02424,     "AMZN": 0.04782,     "GOOGL": -0.03608,     "JPM": -0.01722,     "XOM": 0.00327,     "UNH": 0 ...
[LLM RAW PREVIEW] signal: ```json {   "perspective": "signal",   "proposed_trades": {     "AAPL": 0.0275,     "MSFT": -0.0566,     "NVDA": -0.0850,     "AMZN": -0.0272,     "GOOGL": -0.0850,     "JPM": -0.0137,     "XOM": 0.0275,     "UNH": 0.052 ...
[LLM CALL] committ

##11.CONCLUSION

**Conclusion — From Single-Gate Decisions to Parallel Institutional Judgment**

This notebook marks a clear shift in the project’s progression: we move from “one agent reasoning through a workflow” to a structure that behaves like a real investment organization. The rebalance problem is not framed as a single-answer question. It is framed as a committee process under constraints, where the primary objective is not persuasion but **reviewable convergence**. The architecture you built here is a formalization of how professional teams actually decide: intake, parallel perspective work, deterministic aggregation, and a stage gate that either approves or escalates.

At the structural level, the model is a **Map–Reduce committee** wrapped inside a governed state machine. The intake node establishes the shared “meeting pack” in state: synthetic portfolio weights, constraints, and a market snapshot. Then the graph fans out into parallel committee branches: risk, cost, and signal. Each branch invokes the LLM as a specialized analyst with a narrow mandate and strict JSON outputs. Those reports are merged safely using explicit merge channels in the TypedDict state (the notebook makes concurrency a first-class concept rather than an accident). After that, the reducer node becomes the portfolio manager’s discipline: it aggregates proposals deterministically, enforces constraints (max/min weights, turnover cap, pruning of tiny trades), and computes diagnostic scores such as disagreement. Finally, the gate node plays the role of the investment committee chair: it checks for errors, checks for high disagreement, decides whether a bounded rerun is justified, and otherwise terminates with an explicit decision. The notebook then exports the required audit artifacts—run manifest, graph spec, and final state—so the entire episode is reproducible and reviewable.

This is real progress because the architecture is now **explicitly multi-perspective**. In earlier notebooks, you were building governed control loops, but the topology was primarily serial and centered on a single reasoning stream. Notebook 1 introduced conditional retry loops for missing information: a disciplined way to ask again, bounded and explicit, rather than silently guessing. Notebook 2 introduced a suitability boundary with hard branching and early termination: the system learned to refuse and redirect, which is a governance function, not a cleverness function. Notebook 3 introduced critique loops for evidence gaps in a credit memo: the system learned to self-audit before producing an output. Notebook 4 introduced a tool-augmented node, wrapping a backtest-style computation to keep the LLM from hallucinating numbers. Notebook 5 introduced a stateful regime machine: a structured way to handle execution and liquidity conditions as a controlled state transition problem rather than ad-hoc reasoning.

Notebook 6 extends that trajectory by adding a new architectural dimension: **parallelism with controlled aggregation**. This is the first notebook where concurrency is not just allowed but central to the design. That matters because many real finance decisions are not “one model, one answer.” They are reconciliations: risk versus cost versus signal, compliance versus opportunity, liquidity versus conviction. Serial workflows often hide these tensions by forcing a single voice to speak in one pass. Here, the tensions are surfaced by design, because you generate multiple proposals independently and then measure the distance between them. That distance becomes a governance signal. Disagreement is no longer an invisible human feeling; it becomes a computed metric that can trigger escalation.

The most important conceptual improvement is the division of labor between **LLM creativity and deterministic control**. Earlier notebooks already pushed toward “state drives routing, not text heuristics alone.” This notebook tightens that further: the LLM is used where human judgment belongs—proposal generation under a perspective—while the aggregation and constraint enforcement are deliberately mechanical. This is the institutional pattern you want students to internalize: don’t ask an LLM to be a risk engine, a transaction cost model, and a compliance officer simultaneously. Make it do one job, then enforce structure with deterministic code. The result is faster, safer, and easier to audit.

In governance terms, the notebook also demonstrates maturity: it does not treat failures as exceptions to be hidden. It treats them as state updates to be recorded. Errors and trace events are merge-safe and visible. The system terminates explicitly at END. Loops are bounded and justified. Artifacts are exported by default. This is what it means to build an agentic system for finance: not a chatbot that sounds confident, but a workflow that behaves like a controlled process and produces evidence that it behaved correctly.

So the progress here is not merely “we added a committee.” The progress is that the topology now resembles a professional institution: **multiple independent views, reconciled under rules, with explicit stopping conditions and audit outputs**. That is the difference between a clever demo and a system that can survive contact with a real investment committee.
